In [1]:
import torch
import torch
from torch import nn
import torchvision

# y = None
embed_size = 96

# backbone = torchvision.models.resnet18(num_classes = embed_size)

class lstm(nn.Module):
    def __init__(self, backbone,embed_size = embed_size, in_between = 64, out_features = 4, num_layers = 2, bidirectional = False):
        super(lstm, self).__init__()
        self.embed_size = embed_size
        self.backbone = backbone
        # self.embedding = nn.Embedding(x_grid_size * y_grid_size + 1, embed_size)
        self.rnn = nn.LSTM(input_size = embed_size, hidden_size = 300, batch_first = True, bidirectional = bidirectional, dropout = 0.1, num_layers = num_layers)
        # self.rnn2 = nn.LSTM(input_size = embed_size, hidden_size = 100, batch_first = True, bidirectional = True, dropout = 0.1, num_layers = num_layers)
        self.dropout = nn.Dropout(p = 0.1)
        self.relu = nn.LeakyReLU()
        self.fc1 = nn.Linear(300, 200)
        self.dropout2 = nn.Dropout(p = 0.1)
        self.fc2 = nn.Linear(200, 5)


        # self.keypoint_head_1 = nn.Linear(in_features=embed_size, out_features=in_between)
        # self.keypoint_head_2 = nn.Linear(in_features=in_between, out_features = out_features)

    def forward(self, x):
        # print(self.rnn(x)[1][0].shape)
        # return self.fc(torch.squeeze(self.rnn(x)[1][0], dim=0))
        # print(self.rnn(x)[0].shape)
        # global y
        # y = x
        a, b, c, d, e = x.shape
        features = self.backbone(x.view((a * b, c, d, e))).view((a, b, self.embed_size))
        return self.fc2(self.dropout2(self.relu( \
            self.fc1(self.dropout(self.rnn(features)[0][:, -1, :])))))
         # self.keypoint_head_2(nn.ReLU()(self.keypoint_head_1(features.view((a * b, self.embed_size)))))



c:\Users\29197\miniconda3\envs\566_new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda'
model = lstm(backbone=torchvision.models.resnet18(num_classes = embed_size)).to(device)

In [3]:
MODEL_PATH = '../resnet_with_regression_and_cls_4_12.pt'
model.load_state_dict(torch.load(MODEL_PATH), strict= False)
model.eval()

lstm(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [4]:
import cProfile
input = torch.zeros(size=(1, 16, 3, 224, 224), device= device)
with torch.no_grad():
    cProfile.run('for i in range(100): \n    model(input)')

         77403 function calls (68903 primitive calls) in 1.900 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.005    0.000    1.886    0.019 1973413933.py:29(forward)
        1    0.014    0.014    1.900    1.900 <string>:1(<module>)
      300    0.000    0.000    0.000    0.000 _VF.py:26(__getattr__)
     2000    0.001    0.000    0.001    0.000 __init__.py:31(__get__)
      100    0.000    0.000    0.028    0.000 _jit_internal.py:475(fn)
     1700    0.002    0.000    0.158    0.000 activation.py:101(forward)
      100    0.000    0.000    0.004    0.000 activation.py:774(forward)
     2000    0.010    0.000    0.139    0.000 batchnorm.py:137(forward)
     2000    0.002    0.000    0.003    0.000 batchnorm.py:408(_check_input_dim)
      700    0.001    0.000    0.001    0.000 container.py:194(__iter__)
  700/400    0.003    0.000    0.781    0.002 container.py:202(forward)
     2000    0.003    0.000    1.

In [5]:

class fast_inference(nn.Module):
    def __init__(self, backbone,embed_size = embed_size, in_between = 64, out_features = 4, num_layers = 2, num_frame = 16, bidirectional = False):
        super(fast_inference, self).__init__()
        self.embed_size = embed_size
        self.backbone = backbone
        # self.embedding = nn.Embedding(x_grid_size * y_grid_size + 1, embed_size)
        self.rnn = nn.LSTM(input_size = embed_size, hidden_size = 300, batch_first = True, bidirectional = bidirectional, dropout = 0.1, num_layers = num_layers)
        # self.rnn2 = nn.LSTM(input_size = embed_size, hidden_size = 100, batch_first = True, bidirectional = True, dropout = 0.1, num_layers = num_layers)
        self.dropout = nn.Dropout(p = 0.1)
        self.relu = nn.LeakyReLU()
        self.fc1 = nn.Linear(300, 200)
        self.dropout2 = nn.Dropout(p = 0.1)
        self.fc2 = nn.Linear(200, 5)

        self.features = torch.zeros(size=(num_frame, embed_size), device = device)
        self.num_frame = num_frame


        # self.keypoint_head_1 = nn.Linear(in_features=embed_size, out_features=in_between)
        # self.keypoint_head_2 = nn.Linear(in_features=in_between, out_features = out_features)

        # self.training = False

    def forward(self, x):
        # print(self.rnn(x)[1][0].shape)
        # return self.fc(torch.squeeze(self.rnn(x)[1][0], dim=0))
        # print(self.rnn(x)[0].shape)
        # global y
        # y = x
        o = self.backbone(x)
        
        self.features = torch.concat((self.features[1:,:], o))
        
        # print(self.features.shape)
        
        return self.fc2(self.dropout2(self.relu( \
            self.fc1(self.dropout(self.rnn(self.features)[0][-1, :])))))

In [6]:
fast_model = fast_inference(backbone=torchvision.models.resnet18(num_classes = embed_size)).to(device)
fast_model.load_state_dict(torch.load(MODEL_PATH), strict= False)
fast_model.eval()

fast_inference(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [7]:
# fast_model(torch.zeros(size=(1, 3, 224, 224)))

In [8]:
input = torch.zeros(size=(1, 3, 224, 224), device=device)
with torch.no_grad():
    cProfile.run('for i in range(100): \n    fast_model(input)')

         78703 function calls (70103 primitive calls) in 0.912 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.007    0.000    0.908    0.009 1150694688.py:24(forward)
        1    0.003    0.003    0.912    0.912 <string>:1(<module>)
      300    0.000    0.000    0.000    0.000 _VF.py:26(__getattr__)
     2000    0.001    0.000    0.001    0.000 __init__.py:31(__get__)
      100    0.000    0.000    0.009    0.000 _jit_internal.py:475(fn)
     1700    0.003    0.000    0.049    0.000 activation.py:101(forward)
      100    0.000    0.000    0.004    0.000 activation.py:774(forward)
     2000    0.012    0.000    0.192    0.000 batchnorm.py:137(forward)
     2000    0.003    0.000    0.004    0.000 batchnorm.py:408(_check_input_dim)
      700    0.001    0.000    0.001    0.000 container.py:194(__iter__)
  700/400    0.003    0.000    0.677    0.002 container.py:202(forward)
     2000    0.003    0.000    0.

In [9]:
input = torch.randn(1, 16, 3, 224, 224).to(device)

output1 = model(input)

for i in range(16):
    output2 = fast_model(input[:, i, :, :, :])

print(output2)
print(output1)

tensor([ 0.1757,  0.1765, -0.5562,  0.0133, -0.0063], device='cuda:0',
       grad_fn=<AddBackward0>)
tensor([[ 0.1758,  0.1764, -0.5562,  0.0133, -0.0063]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [10]:
# fast_model.rnn(torch.zeros(size=(16, 96)))[0]

In [11]:
# fast_model.rnn(torch.zeros(size=(16, 96)))[1][1].shape

In [12]:
# fast_model.rnn(torch.zeros(size=(1, 96)))[0][-1, :].shape

In [13]:
# fast_model.rnn(torch.zeros(size=(1, 96)))[0]

In [14]:
from thop import profile
input = torch.randn(1, 16, 3, 224, 224).to(device)
flops, params = profile(model, inputs = (input, ))

print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_lstm() for <class 'torch.nn.modules.rnn.LSTM'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
FLOPs = 29.196472904G
Params = 12.486965M


In [15]:
from thop import profile

input = torch.randn(1, 3, 224, 224).to(device)
flops, params = profile(fast_model, inputs=(input, ))

print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_lstm() for <class 'torch.nn.modules.rnn.LSTM'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
FLOPs = 3.674204744G
Params = 12.486965M
